

Tôi sẽ giải thích cách hoạt động của pipeline RAG (Retrieval Augmented Generation) này:

1. **Khởi tạo và Cấu hình**:
```python
async def on_startup(self):
    # Set OpenAI API key
    os.environ["OPENAI_API_KEY"] = "your_openai_api_key_here"
```

2. **Tạo Document Store và Load Data**:
```python
# Tạo kho lưu trữ document trong bộ nhớ
document_store = InMemoryDocumentStore()

# Load dataset về 7 kỳ quan thế giới
dataset = load_dataset("bilgeyucel/seven-wonders", split="train")
docs = [Document(content=doc["content"], meta=doc["meta"]) for doc in dataset]
```

3. **Xử lý Embeddings**:
```python
# Tạo embeddings cho documents
doc_embedder = SentenceTransformersDocumentEmbedder(
    model="sentence-transformers/all-MiniLM-L6-v2"
)
docs_with_embeddings = doc_embedder.run(docs)
document_store.write_documents(docs_with_embeddings["documents"])

# Tạo embedder cho câu hỏi của user
text_embedder = SentenceTransformersTextEmbedder(
    model="sentence-transformers/all-MiniLM-L6-v2"
)
```

4. **Thiết lập Pipeline**:
```python
# Tạo retriever để tìm documents liên quan
retriever = InMemoryEmbeddingRetriever(document_store)

# Template để tạo prompt
template = """
Given the following information, answer the question.
Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}
Question: {{question}}
Answer:
"""

# Tạo generator sử dụng OpenAI
generator = OpenAIGenerator(model="gpt-3.5-turbo")
```

5. **Kết nối các Components**:
```python
self.basic_rag_pipeline.connect(
    "text_embedder.embedding", "retriever.query_embedding"
)
self.basic_rag_pipeline.connect("retriever", "prompt_builder.documents")
self.basic_rag_pipeline.connect("prompt_builder", "llm")
```

6. **Xử lý Câu hỏi**:
```python
def pipe(self, user_message: str, ...):
    response = self.basic_rag_pipeline.run({
        "text_embedder": {"text": question},
        "prompt_builder": {"question": question},
    })
    return response["llm"]["replies"][0]
```

**Luồng xử lý khi user đặt câu hỏi**:
1. Câu hỏi được chuyển thành vector embedding
2. Vector này được so sánh với embeddings của documents trong store
3. Retriever tìm các documents liên quan nhất
4. Documents này được đưa vào template làm context
5. OpenAI sẽ trả lời dựa trên context và câu hỏi
6. Trả kết quả về cho user

Ví dụ:
- User hỏi: "Tell me about the Great Wall of China"
- Pipeline sẽ tìm documents liên quan đến Great Wall
- OpenAI sẽ tổng hợp thông tin từ documents để trả lời
